In [1]:
import numpy as np
import pandas as pd

In [2]:
mt = pd.read_csv('dataset/kaggle/TourneyCompactResults.csv')
mr = pd.read_csv('dataset/kaggle/RegularSeasonCompactResults.csv')
teams = pd.read_csv('dataset/kaggle/Teams.csv')

pcbrl = list()
for s in range(2002, 2017):
    tmp = pd.read_csv('dataset/kenpom/PCBR_' + str(s) + '.csv')
    tmp['Season'] = s
    pcbrl.append(tmp)

In [3]:
#mt.loc[mt.Season.isin(range(2002,2017))]
pcbr = pd.concat(pcbrl)
pcbr.Season.astype('int')
pcbr = pd.merge(pcbr, teams, how = 'left', left_on = 'Team', right_on='Team_Name')

In [4]:
#pcbr.query('Season == 2002 and Team_Id == 1401')
#for i in range(2002,2017):
#    c = pcbr[pcbr.Season == i]
#    print(c[c.Team_Id.duplicated()])

In [5]:
m = pd.concat([mt,mr])
m.shape

(147339, 8)

In [6]:
m['Team1'] = m.apply(lambda x: x.Wteam if x.Wteam < x.Lteam else x.Lteam, axis=1)
m['Team2'] = m.apply(lambda x: x.Lteam if x.Wteam < x.Lteam else x.Wteam, axis=1)
m['WL'] = m.apply(lambda x: 1 if x.Wteam < x.Lteam else 0, axis=1)

In [7]:
m.head(20)

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Team1,Team2,WL
0,1985,136,1116,63,1234,54,N,0,1116,1234,1
1,1985,136,1120,59,1345,58,N,0,1120,1345,1
2,1985,136,1207,68,1250,43,N,0,1207,1250,1
3,1985,136,1229,58,1425,55,N,0,1229,1425,1
4,1985,136,1242,49,1325,38,N,0,1242,1325,1
5,1985,136,1246,66,1449,58,N,0,1246,1449,1
6,1985,136,1256,78,1338,54,N,0,1256,1338,1
7,1985,136,1260,59,1233,58,N,0,1233,1260,0
8,1985,136,1314,76,1292,57,N,0,1292,1314,0
9,1985,136,1323,79,1333,70,N,0,1323,1333,1


In [8]:
data = m.loc[m.Season.isin(range(2002,2017)), ['Season','Team1','Team2','WL']]
#data = m.loc[m.Season == 2002, ['Season','Team1','Team2','WL']]
data.shape

(76774, 4)

In [9]:
data = pd.merge(data, pcbr, left_on=['Season','Team1'], right_on=['Season', 'Team_Id'], how = 'left')
data.shape

(76774, 19)

In [10]:
data = pd.merge(data, pcbr, left_on=['Season','Team2'], right_on=['Season', 'Team_Id'], how = 'left')

In [11]:
data = data.drop(['Rank_x','Rank_y','Team_x','Team_y','Conf_x','Conf_y','W-L_x','W-L_y',
                  'Team_Id_x','Team_Id_y','Team_Name_x','Team_Name_y','Season','Team1','Team2'], axis=1)
data.columns

Index([u'WL', u'AdjEM_x', u'AdjO_x', u'AdjD_x', u'AdjT_x', u'Luck_x',
       u'SOS AdjEM_x', u'SOS OppO_x', u'SOS OppD_x', u'NCSOS AdjEM_x',
       u'AdjEM_y', u'AdjO_y', u'AdjD_y', u'AdjT_y', u'Luck_y', u'SOS AdjEM_y',
       u'SOS OppO_y', u'SOS OppD_y', u'NCSOS AdjEM_y'],
      dtype='object')

In [12]:
data.rename(columns={'AdjEM_x':'T1_AdjEM',
                     'AdjO_x':'T1_AdjO',
                     'AdjD_x':'T1_AdjD',
                     'AdjT_x':'T1_AdjT',
                     'Luck_x':'T1_Luck',
                     'SOS AdjEM_x':'T1_SOSADjEM',
                     'SOS OppO_x':'T1_SOSOppO',
                     'SOS OppD_x':'T1_SOSOppD',
                     'NCSOS AdjEM_x':'T1_NCSOSAdjEM',
                     'AdjEM_y':'T2_AdjEM',
                     'AdjO_y':'T2_AdjO',
                     'AdjD_y':'T2_AdjD',
                     'AdjT_y':'T2_AdjT',
                     'Luck_y':'T2_Luck',
                     'SOS AdjEM_y':'T2_SOSADjEM',
                     'SOS OppO_y':'T2_SOSOppO',
                     'SOS OppD_y':'T2_SOSOppD',
                     'NCSOS AdjEM_y':'T2_NCSOSAdjEM'}, inplace=True)
data.head()

,WL,T1_AdjEM,T1_AdjO,T1_AdjD,T1_AdjT,T1_Luck,T1_SOSADjEM,T1_SOSOppO,T1_SOSOppD,T1_NCSOSAdjEM,T2_AdjEM,T2_AdjO,T2_AdjD,T2_AdjT,T2_Luck,T2_SOSADjEM,T2_SOSOppO,T2_SOSOppD,T2_NCSOSAdjEM
0,0,-8.69,98.1,106.8,74.2,0.097,-15.24,93.2,108.4,-2.29,-1.20,98.6,99.8,68.9,-0.070,-3.90,98.7,102.6,-1.15
1,1,18.73,111.9,93.2,69.1,0.076,9.90,106.3,96.4,2.49,-2.48,97.5,100.0,71.6,0.061,-5.12,100.3,105.4,-5.36
2,1,20.52,117.2,96.7,73.0,0.080,14.48,109.9,95.4,18.68,8.90,106.1,97.2,64.8,-0.058,0.23,101.4,101.2,2.86
3,1,34.14,118.9,84.7,75.9,-0.027,10.09,107.8,97.7,7.52,-5.33,95.2,100.5,68.0,0.039,-10.81,95.4,106.2,-1.40
4,1,24.80,112.0,87.2,66.8,-0.049,13.66,109.0,95.4,13.51,14.11,109.4,95.2,63.6,0.011,5.87,104.9,99.0,3.57


In [13]:
data.shape

(76774, 19)

In [14]:
data.to_csv('dataset/All_PCBR_For_2002_2016_Matchup.csv', index=False)